In [0]:
display(dbutils.fs.mounts())

mountPoint,source,encryptionType
/databricks-datasets,databricks-datasets,
/Volumes,UnityCatalogVolumes,
/databricks/mlflow-tracking,databricks/mlflow-tracking,
/databricks-results,databricks-results,
/databricks/mlflow-registry,databricks/mlflow-registry,
/Volume,DbfsReserved,
/volumes,DbfsReserved,
/,DatabricksRoot,
/volume,DbfsReserved,


In [0]:
configs = {
  "fs.azure.account.auth.type": "OAuth",
  "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
  "fs.azure.account.oauth2.client.id": "6d985239-6211-4eff-8fc2-8076bf156f12",
  "fs.azure.account.oauth2.client.secret": $secret,
  "fs.azure.account.oauth2.client.endpoint": $endpoint
}

In [0]:
# Setting up the access key for access to the storage container where bronze layer data is stored.

spark.conf.set(
  "fs.azure.account.key.retaildatalakejp.blob.core.windows.net",
   $value
)



In [0]:
df_bronze = spark.read.format("parquet").load(
  "wasbs://bronze@retaildatalakejp.blob.core.windows.net/OnlineRetail.parquet"
)


In [0]:
df_bronze.show(5)

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
InvoiceNo|StockCode| Description|Quantity| InvoiceDate|UnitPrice|CustomerID| Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
 536365| 85123A|WHITE HANGING HEA...| 6|12/1/2010 8:26| 2.55| 17850|United Kingdom|
 536365| 71053| WHITE METAL LANTERN| 6|12/1/2010 8:26| 3.39| 17850|United Kingdom|
 536365| 84406B|CREAM CUPID HEART...| 8|12/1/2010 8:26| 2.75| 17850|United Kingdom|
 536365| 84029G|KNITTED UNION FLA...| 6|12/1/2010 8:26| 3.39| 17850|United Kingdom|
 536365| 84029E|RED WOOLLY HOTTIE...| 6|12/1/2010 8:26| 3.39| 17850|United Kingdom|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
only showing top 5 rows

In [0]:
#Transformations to convert the bronze layer data into silver layer by cleaning it.

from pyspark.sql.functions import *

df_bronze.count()




Out[6]: 541909

In [0]:
#Filtering the data to remove null values and ? from description column 
filtered_descriptions = df_bronze.filter(
    (col("Description").isNotNull()) &
    (~trim(col("Description")).isin("", "?"))
)
filtered_descriptions.count()


Out[7]: 540408

In [0]:
#Filtering dataframe by removing negative values from Quantity column and Unit Price column

filtered_data = filtered_descriptions.filter(
    (col("Quantity") > 0) &
    (col("UnitPrice") > 0)
)
filtered_data.count()

Out[8]: 419211

In [0]:
# Step 5: Filter rows with CustomerID present
filter_customerid_data = filtered_data.filter(col("CustomerID").isNotNull())
filter_customerid_data.count()

Out[9]: 304911

In [0]:
# Step 6: Add derived column - Total Sales
salesamount_df = filter_customerid_data.withColumn("SalesAmount", round(col("Quantity") * col("UnitPrice"), 2))
salesamount_df.show(10)


+---------+---------+--------------------+--------+--------------+---------+----------+--------------+-----------+
InvoiceNo|StockCode| Description|Quantity| InvoiceDate|UnitPrice|CustomerID| Country|SalesAmount|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+-----------+
 536365| 85123A|WHITE HANGING HEA...| 6|12/1/2010 8:26| 2.55| 17850|United Kingdom| 15.3|
 536365| 71053| WHITE METAL LANTERN| 6|12/1/2010 8:26| 3.39| 17850|United Kingdom| 20.34|
 536365| 84406B|CREAM CUPID HEART...| 8|12/1/2010 8:26| 2.75| 17850|United Kingdom| 22.0|
 536365| 84029G|KNITTED UNION FLA...| 6|12/1/2010 8:26| 3.39| 17850|United Kingdom| 20.34|
 536365| 84029E|RED WOOLLY HOTTIE...| 6|12/1/2010 8:26| 3.39| 17850|United Kingdom| 20.34|
 536365| 22752|SET 7 BABUSHKA NE...| 2|12/1/2010 8:26| 7.65| 17850|United Kingdom| 15.3|
 536365| 21730|GLASS STAR FROSTE...| 6|12/1/2010 8:26| 4.25| 17850|United Kingdom| 25.5|
 536366| 22633|HAND WARMER UNION...| 6|12/1/2010 8:28| 1.85| 17850|United Kingdom| 11.1|
 536366| 22632|HAND WARMER RED P...| 6|12/1/2010 8:28| 1.85| 17850|United Kingdom| 11.1|
 536367| 84879|ASSORTED COLOUR B...| 32|12/1/2010 8:34| 1.69| 13047|United Kingdom| 54.08|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+-----------+
only showing top 10 rows

In [0]:
# Step 7: Extract Year, Month, Day, Hour from InvoiceDate
timestamp_df = salesamount_df.withColumn("InvoiceDateTS", to_timestamp(col("InvoiceDate"), "M/d/yyyy H:mm"))

# Step 2: Extract components
timestamp_df = timestamp_df.withColumn("InvoiceYear", year(col("InvoiceDateTS"))) \
       .withColumn("InvoiceMonth", month(col("InvoiceDateTS"))) \
       .withColumn("InvoiceDay", dayofmonth(col("InvoiceDateTS"))) \
       .withColumn("InvoiceHour", hour(col("InvoiceDateTS")))
timestamp_df.show(10)

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+-----------+-------------------+-----------+------------+----------+-----------+
InvoiceNo|StockCode| Description|Quantity| InvoiceDate|UnitPrice|CustomerID| Country|SalesAmount| InvoiceDate|InvoiceYear|InvoiceMonth|InvoiceDay|InvoiceHour|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+-----------+-------------------+-----------+------------+----------+-----------+
 536365| 85123A|WHITE HANGING HEA...| 6|12/1/2010 8:26| 2.55| 17850|United Kingdom| 15.3|2010-12-01 08:26:00| 2010| 12| 1| 8|
 536365| 71053| WHITE METAL LANTERN| 6|12/1/2010 8:26| 3.39| 17850|United Kingdom| 20.34|2010-12-01 08:26:00| 2010| 12| 1| 8|
 536365| 84406B|CREAM CUPID HEART...| 8|12/1/2010 8:26| 2.75| 17850|United Kingdom| 22.0|2010-12-01 08:26:00| 2010| 12| 1| 8|
 536365| 84029G|KNITTED UNION FLA...| 6|12/1/2010 8:26| 3.39| 17850|United Kingdom| 20.34|2010-12-01 08:26:00| 2010| 12| 1| 8|
 536365| 84029E|RED WOOLLY HOTTIE...| 6|12/1/2010 8:26| 3.39| 17850|United Kingdom| 20.34|2010-12-01 08:26:00| 2010| 12| 1| 8|
 536365| 22752|SET 7 BABUSHKA NE...| 2|12/1/2010 8:26| 7.65| 17850|United Kingdom| 15.3|2010-12-01 08:26:00| 2010| 12| 1| 8|
 536365| 21730|GLASS STAR FROSTE...| 6|12/1/2010 8:26| 4.25| 17850|United Kingdom| 25.5|2010-12-01 08:26:00| 2010| 12| 1| 8|
 536366| 22633|HAND WARMER UNION...| 6|12/1/2010 8:28| 1.85| 17850|United Kingdom| 11.1|2010-12-01 08:28:00| 2010| 12| 1| 8|
 536366| 22632|HAND WARMER RED P...| 6|12/1/2010 8:28| 1.85| 17850|United Kingdom| 11.1|2010-12-01 08:28:00| 2010| 12| 1| 8|
 536367| 84879|ASSORTED COLOUR B...| 32|12/1/2010 8:34| 1.69| 13047|United Kingdom| 54.08|2010-12-01 08:34:00| 2010| 12| 1| 8|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+-----------+-------------------+-----------+------------+----------+-----------+
only showing top 10 rows

In [0]:
final_df = timestamp_df.withColumn("Description", trim(col("Description"))) \
                       .withColumn("Country", trim(col("Country")))

final_df.show(2)

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+-----------+-------------------+-----------+------------+----------+-----------+
InvoiceNo|StockCode| Description|Quantity| InvoiceDate|UnitPrice|CustomerID| Country|SalesAmount| InvoiceDate|InvoiceYear|InvoiceMonth|InvoiceDay|InvoiceHour|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+-----------+-------------------+-----------+------------+----------+-----------+
 536365| 85123A|WHITE HANGING HEA...| 6|12/1/2010 8:26| 2.55| 17850|United Kingdom| 15.3|2010-12-01 08:26:00| 2010| 12| 1| 8|
 536365| 71053| WHITE METAL LANTERN| 6|12/1/2010 8:26| 3.39| 17850|United Kingdom| 20.34|2010-12-01 08:26:00| 2010| 12| 1| 8|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+-----------+-------------------+-----------+------------+----------+-----------+
only showing top 2 rows

In [0]:
final_df.printSchema()

root
-- InvoiceNo: string (nullable = true)
-- StockCode: string (nullable = true)
-- Description: string (nullable = true)
-- Quantity: string (nullable = true)
-- InvoiceDate: string (nullable = true)
-- UnitPrice: string (nullable = true)
-- CustomerID: string (nullable = true)
-- Country: string (nullable = true)
-- SalesAmount: double (nullable = true)
-- InvoiceDate: timestamp (nullable = true)
-- InvoiceYear: integer (nullable = true)
-- InvoiceMonth: integer (nullable = true)
-- InvoiceDay: integer (nullable = true)
-- InvoiceHour: integer (nullable = true)

In [0]:
final_df = final_df.drop('InvoiceDate')
final_df.printSchema()

root
-- InvoiceNo: string (nullable = true)
-- StockCode: string (nullable = true)
-- Description: string (nullable = true)
-- Quantity: string (nullable = true)
-- UnitPrice: string (nullable = true)
-- CustomerID: string (nullable = true)
-- Country: string (nullable = true)
-- SalesAmount: double (nullable = true)
-- InvoiceYear: integer (nullable = true)
-- InvoiceMonth: integer (nullable = true)
-- InvoiceDay: integer (nullable = true)
-- InvoiceHour: integer (nullable = true)

In [0]:
#Write data to silver layer
spark.conf.set(
  "fs.azure.account.key.retaildatalakejp.blob.core.windows.net",
  $key
)


final_df.write.mode("overwrite").parquet("wasbs://silver@retaildatalakejp.blob.core.windows.net/OnlineRetailCleaned.parquet")